1.SetupEnvironment:Installrequired Python libraries.

In [1]:
!pip install pandas numpy faker openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.0 MB/s eta 0:00:00


pandas → data manipulation

numpy → numeric operations

faker → generate realistic Indian fake data

openpyxl → export Excel files

2.Generate Raw CSV Files : Create 4 raw datasets (customers, products, orders, order_items).

In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker('en_IN')  # Indian locale

# 1️. Customers Dataset
num_customers = 5000
customer_id = [f"CUST{1000+i}" for i in range(num_customers)]
customer_name = [fake.name() for _ in range(num_customers)]
city = [random.choice(['Hyderabad', 'Bengaluru', 'Chennai', 'Pune', 'Mumbai']) for _ in range(num_customers)]
gender = [random.choice(['Male', 'Female', 'Other']) for _ in range(num_customers)]
age = [random.randint(18, 65) for _ in range(num_customers)]
customers = pd.DataFrame({'customer_id': customer_id,'customer_name': customer_name,'city': city,'gender': gender,'age': age})
customers.to_csv('customers.csv', index=False)

# 2️. Products Dataset
num_products = 300
product_id = [f"PROD{1000+i}" for i in range(num_products)]
product_name = [fake.word().title() for _ in range(num_products)]
category = [random.choice(['Electronics','Fashion','Home','Groceries']) for _ in range(num_products)]
price = [round(random.uniform(100,5000),2) for _ in range(num_products)]
products = pd.DataFrame({'product_id': product_id,'product_name': product_name,'category': category,'price': price})
products.to_csv('products.csv', index=False)

# 3️. Orders Dataset
num_orders = 15000
order_id = [f"ORD{10000+i}" for i in range(num_orders)]
order_customer_id = [random.choice(customer_id) for _ in range(num_orders)]
order_date = [fake.date_between(start_date='-3y', end_date='today') for _ in range(num_orders)]
payment_method = [random.choice(['UPI','NetBanking','Card','COD']) for _ in range(num_orders)]
orders = pd.DataFrame({'order_id': order_id,'customer_id': order_customer_id,'order_date': order_date,'payment_method': payment_method})
orders.to_csv('orders.csv', index=False)

# 4️.Order Items Dataset
num_order_items = 22000
order_item_id = [f"ITEM{100000+i}" for i in range(num_order_items)]
order_item_order_id = [random.choice(order_id) for _ in range(num_order_items)]
order_item_product_id = [random.choice(product_id) for _ in range(num_order_items)]
quantity = [random.randint(1,10) for _ in range(num_order_items)]
unit_price = [float(products.loc[products['product_id']==pid,'price']) for pid in order_item_product_id]
total_price = [round(quantity[i]*unit_price[i],2) for i in range(num_order_items)]
order_items = pd.DataFrame({'order_item_id': order_item_id,'order_id': order_item_order_id,'product_id': order_item_product_id,'quantity': quantity,'unit_price': unit_price,'total_price': total_price})
order_items.to_csv('order_items.csv', index=False)

print(" All 4 raw CSV files created successfully!")


/tmp/ipython-input-2770942618.py:42: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  unit_price = [float(products.loc[products['product_id']==pid,'price']) for pid in order_item_product_id]


 All 4 raw CSV files created successfully!


3.Load CSV Files: Read raw CSV files into Python for processing.

In [3]:
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')
order_items = pd.read_csv('order_items.csv')
customers.head()

,customer_id,customer_name,city,gender,age
0,CUST1000,Chanchal Memon,Hyderabad,Male,54
1,CUST1001,Dipta Nath,Hyderabad,Other,31
2,CUST1002,Raksha Ganesan,Hyderabad,Other,54
3,CUST1003,Aarini Singh,Pune,Female,23
4,CUST1004,Jack Deo,Hyderabad,Female,29


4.Merge & Clean Data : Combine all datasets into a single full_cleaned_data.csv for analysis.

In [5]:
# Merge order_items with orders
order_items_orders = pd.merge(order_items, orders, on='order_id', how='left')
# Merge with products
order_items_orders_products = pd.merge(order_items_orders, products, on='product_id', how='left')
# Merge with customers
full_data = pd.merge(order_items_orders_products, customers, on='customer_id', how='left')
# Convert order_date to datetime
full_data['order_date'] = pd.to_datetime(full_data['order_date'])
# Save cleaned dataset
full_data.to_csv('full_cleaned_data.csv', index=False)
print(" Full cleaned dataset saved as full_cleaned_data.csv")

 Full cleaned dataset saved as full_cleaned_data.csv


5.Exploratory Data Analysis (EDA) : Get insights on sales, products, categories, and regions.

In [6]:
# 1️. Monthly Sales
monthly_sales = full_data.groupby(full_data['order_date'].dt.to_period('M'))['total_price'].sum()
print("Monthly Sales:\n", monthly_sales.head())
# 2️. Revenue by Category
category_revenue = full_data.groupby('category')['total_price'].sum()
print("\nRevenue by Category:\n", category_revenue)
# 3️. Top 10 Products
top_products = full_data.groupby('product_name')['total_price'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 Products:\n", top_products)
# 4️. Sales by City
city_sales = full_data.groupby('city')['total_price'].sum()
print("\nSales by City:\n", city_sales)

Monthly Sales:
 order_date
2022-11    1721040.47
2022-12    8607281.79
2023-01    8360909.33
2023-02    6554958.75
2023-03    9139833.03
Freq: M, Name: total_price, dtype: float64

Revenue by Category:
 category
Electronics    57724927.21
Fashion        82387114.65
Groceries      80064439.77
Home           84035833.99
Name: total_price, dtype: float64

Top 10 Products:
 product_name
Ex            5634318.20
Itaque        5246633.74
Maiores       5167598.60
Occaecati     5059322.16
Blanditiis    4934303.74
Laborum       4890228.66
Voluptatem    4536488.67
Magni         4530549.10
Nam           4384422.77
Labore        4337782.18
Name: total_price, dtype: float64

Sales by City:
 city
Bengaluru    60401383.80
Chennai      61001844.10
Hyderabad    54677722.37
Mumbai       64048808.71
Pune         64082556.64
Name: total_price, dtype: float64


6.Export Cleaned Data for Excel/Power BI: Save the final dataset for visualization dashboards.

In [7]:
# Export to CSV
full_data.to_csv('full_cleaned_data.csv', index=False)
# Export to Excel
full_data.to_excel('Retail_Data_Cleaned.xlsx', index=False)
print(" Cleaned dataset exported for Excel/Power BI")

 Cleaned dataset exported for Excel/Power BI


8.Download CSV / Excel from Colab:

In [8]:
from google.colab import files
files.download('customers.csv')
files.download('products.csv')
files.download('orders.csv')
files.download('order_items.csv')
files.download('full_cleaned_data.csv')
files.download('Retail_Data_Cleaned.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>